<a href="https://colab.research.google.com/github/Alpha25-ai/Smart-cooking-assistant/blob/main/cooking_assistant_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install gTTs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [12]:
!pip install fuzzywuzzy

In [10]:
!pip install gradio

In [15]:
import pandas as pd

db = pd.read_csv("/recipes_ingredients.csv",on_bad_lines = "skip", engine = "python")

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
db["text"] = db["Name"].fillna("") + " " + db["Ingredients"].fillna("") + " " + db["Steps"].fillna("")

In [17]:
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(db["text"])

In [18]:
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(db["text"])

def search_recipes(query, top_n=3):
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf_matrix).flatten()
    indices = similarity.argsort()[::-1][:top_n]

    results = []
    for i in indices:
        results.append({
            "Name": db.iloc[i]["Name"],
            "Description": db.iloc[i]["Description"],
            "Ingredients": db.iloc[i]["Ingredients"],
            "Ingredients_raw": db.iloc[i]["Ingredients_raw"],
            "Steps": db.iloc[i]["Steps"],
            "Servings": db.iloc[i]["Servings"],
            "Serving_size": db.iloc[i]["Serving_size"],
            "Tags": db.iloc[i]["Tags"],
            "score": similarity[i]
        })
    return results
print("Smart Cooking Assistant Ready! (type 'exit' to quit)")

while True:
    query = input("\nWhat do you want to cook? ")
    if query.lower() == "exit":
        break

    results = search_recipes(query)
    if results:
        print("\nTop Recipes:")
        for r in results:
            print(f"\n {r['Name']} (score: {r['score']:.2f})")
            print("Description:",r["Description"][:200],"...")
            print("Ingredients:", r["Ingredients"])
            print("Ingredients_raw:", r["Ingredients_raw"])
            print("Steps:", r["Steps"][:200], "...")
            print("Servings",r["Servings"])
            print("Serving_size",r["Serving_size"])
            print("Tags", r["Tags"][:200], "...")

    else:
        print("Sorry, no recipe found.")


Smart Cooking Assistant Ready! (type 'exit' to quit)

What do you want to cook? cake

Top Recipes:

 Cake Mix Chocolate Mayonnaise Cake (score: 0.82)
Description: I like to get creative with boxed cake mixes. It's more convenient and less time consuming than making cakes from scratch. The mayonnaise in this cake makes it really moist. You cannot tell the mayonn ...
Ingredients: ["duncan devil  s food cake mix", "mayonnaise", "chocolate chips", "icing powdered sugar"]
Ingredients_raw: ["1   box   duncan hines devil\u0027s food cake mix (or any brand)","2   large    eggs","1 1/3  cups    water","1   cup    mayonnaise","1   cup    chocolate chips","  icing, of choice  or   powdered sugar"]
Steps: ["Preheat oven to 350.", "Grease and flour a bundt cake pan or a 9x13x2 inch cake pan.", "Dump the cake mix into a large mixing bowl.", "Add eggs, water and mayonnaise.", "Mix by hand or with an elect ...
Servings 1.0
Serving_size 1 (1108 g)
Tags ["60-minutes-or-less", "time-to-make", "course", "

In [21]:
import pandas as pd
from fuzzywuzzy import process
import gradio as gr

db = pd.read_csv("/recipes_ingredients.csv", on_bad_lines="skip", engine="python")


def find_recipe(query):

    choices = db['Name'].dropna().tolist()
    best_match, score = process.extractOne(query, choices)

    if score > 60:
        result = db[db['Name'] == best_match].iloc[0]
        return best_match, result["Ingredients"], result["Steps"],result["Description"],result["Ingredients_raw"],result["Servings"],result["Serving_size"],result["Tags"], score
    else:
        return None, None, None, None, None, None, None, None, None

print("RecipeBot: Hi! Ask me about any recipe (type 'exit' to quit)")

while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("RecipeBot: Goodbye! Happy cooking! 🍲")
        break

    name, Ingredients, Steps, Description, Ingredients_raw, Servings, Serving_size, Tags, score = find_recipe(query)

    if Ingredients:
        print(f"\n RecipeBot: I think you meant *{name}* (match score: {score})")
        print("\n Ingredients:\n", Ingredients)
        print("\n Steps:\n", Steps[:400], "...")
        print("Description:",Description[:200],"...")
        print("Ingredients_raw:", Ingredients_raw)
        print("Servings",Servings)
        print("Serving_size",Serving_size)
        print("Tags", Tags[:200], "...")
    else:
        print(" RecipeBot: Sorry, I couldn't find a close match. Try another one!")


RecipeBot: Hi! Ask me about any recipe (type 'exit' to quit)

You: burger

 RecipeBot: I think you meant *Faye's Broiled Hamburgers* (match score: 90)

 Ingredients:
 ["ground beef", "onion", "boiling water", "beef bouillon powder oxo sodium content", "Worcestershire sauce", "steak sauce", "ketchup"]

 Steps:
 ["Dissolve the beef bouillon in the water.  Allow to cool.", "Mix all ingredients except the beef, in large bowl.", "Add the ground beef and mix lightly but very thoroughly.", "Shape into patties 3/4" thick.", "Fry, broil or grill as desired."] ...
Description: Friends love them - I hope you do too.If freezing, put layer of parchment paper between each patty and wrap tightly.  Cook from frozen.Can be frozen for up to 3 months.The beef bouillon you use will m ...
Ingredients_raw: ["5   lbs    lean ground beef","1   teaspoon    pepper","1   cup    onion, chopped fine ","1 1/2  tablespoons    boiling water","5   teaspoons    beef bouillon powder (I prefer oxo due sodium content)","1

In [33]:
import gradio as gr
import pandas as pd
import re

def chatbot(message, history):
    try:
        msg = str(message).lower()

        words = re.findall(r"\b\w+\b", msg)
        ingredients = [w for w in words if len(w) > 3]

        if not ingredients:
            return "Please mention at least one ingredient."

        mask = db["Ingredients"].astype(str).str.lower()
        matched = db[mask.apply(lambda x: any(ing in x for ing in ingredients))]

        if not matched.empty:
            first_recipe = matched.iloc[0]
            reply = f" You can make **{first_recipe['Name']}**!\n\n"
            reply += f" **Ingredients:**\n{first_recipe['Ingredients']}\n\n"
            reply += f" **Steps:**\n{first_recipe['Steps']}"
            return reply

        else:
            return f"⚠️ Sorry, I couldn't find any recipe with {', '.join(ingredients)}."

    except Exception as e:
        return f"⚠️ Internal Error: {str(e)}"

demo = gr.ChatInterface(
    fn=chatbot,
    title="Recipe Chatbot",
    description="What do you want to cook?"
)

demo.launch()


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://88f901fb951ec7b224.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
